In [1]:
import tensorflow as tf
import numpy as np
import os
import pathlib
from deepsudoku import load_supervised_data, metrics_and_callbacks, REPO_PATH

os.chdir(REPO_PATH)

BATCHSIZE = 256 * 4

2023-07-26 18:53:26.192846: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 18:53:27.359540: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model = tf.keras.models.load_model(
    "ckpt/final_model.ckpt",
    custom_objects={"SudokuWinRate": metrics_and_callbacks.SudokuWinRate}
)

2023-07-26 18:53:42.539756: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-26 18:53:42.788186: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-26 18:53:42.788346: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [16]:
test_data_by_difficulty = load_supervised_data.get_data_by_difficulty("test")

metric = metrics_and_callbacks.SudokuWinRate()

results = {}

for difficulty, data in test_data_by_difficulty.items():

    metric.reset_state()

    preprocessed = load_supervised_data.get_tf_dataset(*data).batch(BATCHSIZE)

    preds = model.predict(preprocessed, verbose = False)

    labels = data[1] - 1

    metric.update_state(labels, preds)

    results[difficulty] = metric.result().numpy()

results

2023-07-26 19:00:48.619183: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [25000,9,9]
	 [[{{node Placeholder/_1}}]]
2023-07-26 19:00:50.183578: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [53500,9,9]
	 [[{{node Placeholder/_1}}]]
2023-07-26 19:00:53.484622: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shap

{'easy': 1.0,
 'medium': 0.9998131,
 'hard': 0.99868,
 'extreme': 0.99531037,
 'insane': 0.98966}